# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/home/pika/App/miniconda3/envs/transformers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据

In [2]:
import pandas as pd

data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
data.head()

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


In [3]:
data = data.dropna()
data.shape

(7765, 2)

## Step3 创建Dataset

In [4]:
from torch.utils.data import Dataset


class MyDataset(Dataset):

    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]

    def __len__(self):
        return len(self.data)

In [5]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## Step4 划分数据集

In [6]:
from torch.utils.data import random_split


trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [7]:
for i in range(10):
    print(trainset[i])

('酒店看起来设施都比较陈旧，房间里居然没有中控，非常不方便。热水到还不错。前台态度一般.', 0)
('酒店员工都很亲切随时报以笑容，但是酒店不算新了，早餐品种单一，每天都一样，晚上的自助晚餐菜品也一般，信价比不高哦，还不如打车去滨海路的餐厅吃，好吃又便宜。酒店spa,健身中心，大堂都非常的普通，游泳池也不算大，不过酒店对面就是私人海滩，可以在海滩边消磨一天，还是不错的。', 1)
('天清岛度假酒店，自然风景很好，优于天元度假村。位置偏僻，适合团队和自驾游。设施一般，相比天元，差距很大。早餐足够丰盛，质量也很好。酒店自制的面包和小点心非常好吃，尤其推荐枣泥蛋糕。早餐培根特别好，广受欢迎，水果的品质也不错。中餐厅的正餐就没什么特色了，味道一般，好在价格没有贵得离谱。开元的中餐厅味道确实很好，除了鱼头，我们对那个过油四季豆印象很深，味道独特，很好吃。关于房间：天清岛酒店共9个楼。1-8号为一个整体，呈蛇形排列，共5层，1层是停车库，2-5层客房。大堂位于中间，左侧是1、2号，右侧依次是3、4、5、6、7、8号。9号楼在岛上是看不到的，位于停车坪和草坪的下面，只有2层，紧邻湖水，要在水上或山对面才能看见正面。1-9号所有房间都是景观房，有观景阳台。房间景观首推9号楼，阳台下面就是湖水，远处是岛屿和层峦，视线开阔无遮拦，有水墨画的写意韵味。“缺点”是有点单调。这也是天清岛和开元最大的差别所在。开元胜在装修和园林景观，精致豪华且丰富多彩，天清岛胜在天水合一的自然景观，没有人工雕饰，但略显单调。选择哪里，要看个人喜好。9号楼的阳台上有茶几和太阳椅，坐在外面看风景、扮风景时，别忘了带个望远镜。我们订到一天豪华房，第二天在房间一直耗到中午才离开。冲杯咖啡，在阳台上看湖面驶过的游船和快艇，再望望远山，非常悠闲惬意。4、5号楼的景观与9号不相上下，面向辽阔的湖水，视线最为开阔，且站的高看的远。有人说隔条马路才是湖面，我觉得这不是问题，因为楼前有酒店的园林绿化，反而丰富了景观效果。这里特别提醒，如果不是一定要睡大床，酒店2张1米2宽单人床的标准间最为实惠。因为湖景大床房全部安排在景观不佳的6号楼，房间大小、设施没有区别，价格却高于其他楼的湖景标间，不划算。之后依次是7、3、6、8号。所谓景观不佳，就是视角逊色、不够开阔。我们第二晚换到6号楼大床房，正对一个小水湾。看出去青山碧水，也很

## Step5 创建Dataloader

In [8]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")


# 没在dataset里处理：tokenizer在batch处理是最快的
def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(
        texts,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )  # dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

    inputs["labels"] = torch.tensor(labels)
    return inputs

In [9]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(
    validset, batch_size=64, shuffle=False, collate_fn=collate_func
)

In [10]:
next(enumerate(validloader))[1]  # 这个时候调用collate_func

{'input_ids': tensor([[ 101, 3302, 1218,  ...,    0,    0,    0],
        [ 101, 1914, 2399,  ...,    0,    0,    0],
        [ 101, 1762, 6935,  ...,  511,  852,  102],
        ...,
        [ 101, 3680, 3613,  ..., 1184, 6842,  102],
        [ 101, 2791, 7313,  ...,    0,    0,    0],
        [ 101, 2523, 3173,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1,
        1, 1, 1

## Step6 创建模型及优化器

In [11]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

if torch.cuda.is_available():
    model = model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
optimizer = Adam(model.parameters(), lr=2e-5)

## Step7 训练与验证

In [13]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)


def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(
                    f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}"
                )
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")

## Step8 模型训练

In [14]:
train()

ep: 0, global_step: 0, loss: 0.7017712593078613
ep: 0, global_step: 100, loss: 0.24556273221969604
ep: 0, global_step: 200, loss: 0.3049444258213043
ep: 0, acc: 0.8981958627700806
ep: 1, global_step: 300, loss: 0.19269973039627075
ep: 1, global_step: 400, loss: 0.21679648756980896
ep: 1, acc: 0.9033504724502563
ep: 2, global_step: 500, loss: 0.15411913394927979
ep: 2, global_step: 600, loss: 0.057508256286382675
ep: 2, acc: 0.905927836894989


## Step9 模型预测

In [15]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [16]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [17]:
pipe(sen)

[{'label': '好评！', 'score': 0.9969767332077026}]